In [2]:
# Importar las bibliotecas necesarias
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from skimage.feature import hog
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import csv
import concurrent.futures

# Definir la ruta de la carpeta con las imágenes
#path = '/content/drive/MyDrive/Estudios/Máster IIR/Segundo cuatrimestre/Proyecto VAI-AAI/chest_xray'
path = '../data'

# Categorías
categorias = ['NORMAL', 'PNEUMONIA']

# Lista de hiperparámetros HOG para probar
combinaciones_hog = [
    #{'orientations': 4, 'pixels_per_cell': (8, 8)},
    #{'orientations': 5, 'pixels_per_cell': (8, 8)},
    #{'orientations': 6, 'pixels_per_cell': (8, 8)},
    {'orientations': 7, 'pixels_per_cell': (8, 8)},
    #{'orientations': 8, 'pixels_per_cell': (8, 8)},
    #{'orientations': 9, 'pixels_per_cell': (8, 8)},
]

total_data = []

def procesar_imagen(imagen_path, hog_params, size, etiqueta):
    imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f'No se pudo leer la imagen: {imagen_path}')
        return None, None

    imagen = cv2.resize(imagen, size)
    fd = hog(imagen, orientations=hog_params['orientations'], pixels_per_cell=hog_params['pixels_per_cell'],
             cells_per_block=(1, 1), visualize=False)

    return fd, etiqueta

def cargar_datos(ruta, hog_params, size=(128, 128)):
    datos = []
    etiquetas = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i, cat in enumerate(categorias):
            carpeta = join(ruta, cat)
            archivos = [f for f in listdir(carpeta) if isfile(join(carpeta, f))]
            for archivo in archivos:
                imagen_path = join(carpeta, archivo)
                futures.append(executor.submit(procesar_imagen, imagen_path, hog_params, size, i))

        for future in concurrent.futures.as_completed(futures):
            fd, label = future.result()
            if fd is not None:
                datos.append(fd)
                etiquetas.append(label)

    if len(datos) == 0 or len(etiquetas) == 0:
        raise ValueError("No se pudieron cargar los datos. Asegúrate de que la ruta y las categorías son correctas.")

    return np.array(datos), np.array(etiquetas)


for hog_params in combinaciones_hog:
  # Cargar datos de entrenamiento, validación y prueba
  datos_entrenamiento, etiquetas_entrenamiento = cargar_datos(join(path, 'train'), hog_params)
  datos_prueba, etiquetas_prueba = cargar_datos(join(path, 'test'), hog_params)

  # Asegurarse de que hay datos para al menos dos clases
  if len(np.unique(etiquetas_entrenamiento)) < 2 or len(np.unique(etiquetas_prueba)) < 2:
      raise ValueError("Se debe cargar datos para al menos dos clases en cada conjunto de datos.")

  # Crear y entrenar el modelo RF
  modelo = RandomForestClassifier(random_state=42, n_jobs=-1)
  modelo.fit(datos_entrenamiento, etiquetas_entrenamiento)

  # Evaluar el modelo
  etiquetas_predichas = modelo.predict(datos_prueba)
  exactitud = metrics.accuracy_score(etiquetas_prueba, etiquetas_predichas)

  # Obtener la matriz de confusión
  cm = confusion_matrix(etiquetas_prueba, etiquetas_predichas)

  # Calcular Sensibilidad (Recall) y Especificidad
  sensibilidad = recall_score(etiquetas_prueba, etiquetas_predichas)
  especificidad = cm[0,0] / (cm[0,0] + cm[0,1])

  # Calcular Precisión y F1-Score
  precision = precision_score(etiquetas_prueba, etiquetas_predichas)
  f1 = f1_score(etiquetas_prueba, etiquetas_predichas)

  # Calcular AUC
  # AUC requiere las puntuaciones o probabilidades de clase positiva, no las etiquetas predichas.
  # Asegúrate de que tu modelo soporte `decision_function` o `predict_proba`.
  if hasattr(modelo, "decision_function"):
      scores = modelo.decision_function(datos_prueba)
  else:
      scores = modelo.predict_proba(datos_prueba)[:, 1]
  auc = roc_auc_score(etiquetas_prueba, scores)


  current_data = {
    'Exactitud': exactitud * 100,
    'Sensibilidad': sensibilidad * 100,
    'Especificidad': especificidad * 100,
    'Precisión': precision * 100,
    'F1-Score': f1,
    'AUC': auc
  }

  total_data.append(current_data)

  nombre_archivo_csv = "../results/hog_combination_rf.csv"


  with open(nombre_archivo_csv, 'a+', newline='') as archivo:
      es_vacio = archivo.tell() == 0
      escritor = csv.writer(archivo)

      # Suponiendo que es_vacio indica si debemos escribir los encabezados
      if es_vacio:
          # Escribir los nombres de las claves del diccionario y luego los nombres de las métricas
          escritor.writerow(list(hog_params.keys()) + list(current_data.keys()))

      # Escribir los valores del diccionario y luego los valores de las métricas
      escritor.writerow(list(hog_params.values()) + list(current_data.values()))

  print("Datos guardados en el archivo CSV.")
  print("Claves:", list(hog_params.keys()) + list(current_data.keys()))
  print("Valores:", list(hog_params.values()) + list(current_data.values()))

  if len(total_data) == len(combinaciones_hog):
    # Encuentra el índice del mejor F1-Score
    index_mejor_f1 = max(range(len(total_data)), key=lambda i: total_data[i]['F1-Score'])

    # Encuentra la configuración HOG correspondiente al mejor F1-Score
    hog_correspondiente = combinaciones_hog[index_mejor_f1]

    for i in range(6, 10):
      data = {
        'orientations': hog_correspondiente['orientations'],
        'pixels_per_cell': (i, i)
      }
      combinaciones_hog.append(data)



Datos guardados en el archivo CSV.
Claves: ['orientations', 'pixels_per_cell', 'Exactitud', 'Sensibilidad', 'Especificidad', 'Precisión', 'F1-Score', 'AUC']
Valores: [7, (8, 8), 71.31410256410257, 98.97435897435898, 25.213675213675213, 68.80570409982175, 0.8117770767613038, 0.8904010519395136]
Datos guardados en el archivo CSV.
Claves: ['orientations', 'pixels_per_cell', 'Exactitud', 'Sensibilidad', 'Especificidad', 'Precisión', 'F1-Score', 'AUC']
Valores: [7, (6, 6), 70.67307692307693, 98.97435897435898, 23.504273504273502, 68.31858407079646, 0.8083769633507853, 0.8763423186500109]
Datos guardados en el archivo CSV.
Claves: ['orientations', 'pixels_per_cell', 'Exactitud', 'Sensibilidad', 'Especificidad', 'Precisión', 'F1-Score', 'AUC']
Valores: [7, (7, 7), 72.11538461538461, 99.23076923076923, 26.923076923076923, 69.35483870967742, 0.8164556962025317, 0.8792844619767697]
Datos guardados en el archivo CSV.
Claves: ['orientations', 'pixels_per_cell', 'Exactitud', 'Sensibilidad', 'Especi